## 1.0 Bibliotecas

In [ ]:
import unidecode
import pandas as pd
from os import listdir
from os.path import isfile, join
from IPython.display import HTML
import seaborn as sns

### 2.0 Funções

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

### 2.0 Constantes

In [ ]:
cols = ['Total', 'Pelo menos uma',
                  'Visual-Não consegue de modo algum', 'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
                  'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade', 'Auditiva-Alguma dificuldade',
                  'Motora-Não consegue de modo algum', 'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade',
                  'Mental', 'Nenhuma', 'Estado']

cols_order = ['Estado', 'Total', 'Pelo menos uma',
                  'Visual-Não consegue de modo algum', 'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
                  'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade', 'Auditiva-Alguma dificuldade',
                  'Motora-Não consegue de modo algum', 'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade',
                  'Mental', 'Nenhuma']

### 3.0 Leitura de Dados

In [ ]:
df_final = pd.DataFrame(columns=cols_order)

for estado in listdir('data'):
    url = 'data\\{}\\Tabela {}.3.1.xls'
    
    file_prefix = listdir('data\\{}'.format(estado))[0][7:11]
    file_prefix = file_prefix if file_prefix[-1] != '.' else file_prefix[:-1]
    
    url = url.format(estado,file_prefix)
    
    df = pd.read_excel(url, skiprows=7, header=None)
    df.drop(columns=[0,9], inplace=True)
    df['Estado'] = estado
    
    df.columns = cols
    df = df[cols_order]
    
    df_final = df_final.append(df.iloc[0])

In [ ]:
df_final.select_dtypes('float64').columns

## 4.0 Conversão de Tipo

In [ ]:
for c in ['Total', 'Pelo menos uma', 'Visual-Não consegue de modo algum',
       'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
       'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade',
       'Auditiva-Alguma dificuldade', 'Motora-Não consegue de modo algum',
       'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade', 'Mental',
       'Nenhuma']:
    df_final[c] = df_final[c].astype('int')

### 4.0 Engenharia de Features

#### 4.1 Total de pessoas c/ repetição entre pessoas com mais de uma deficiência

In [ ]:
df_final['Total c/ Repetição'] = df_final.iloc[:, 3:].apply(lambda x: x.sum(), axis=1)

#### 4.2 Total de pessoas c/ deficiência c/ repetição entre pessoas com mais de uma deficiência

In [ ]:
df_final['Total c/ Deficiência c/ Repetição'] = df_final.iloc[:, 3:-2].apply(lambda x: x.sum(), axis=1)

#### 4.3 Percentual de pessoas com deficiência

In [ ]:
df_final['% respostas com deficiência'] = round(df_final['Total c/ Deficiência c/ Repetição']/df_final['Total c/ Repetição'],4)*100

In [ ]:
df_final['% pessoas com deficiência'] = round(df_final['Pelo menos uma']/df_final['Total'],4)*100

### 5.4 Coluna UF

In [ ]:
d_uf = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}
d_uf = {unidecode.unidecode(v): k for k, v in d_uf.items()}

In [ ]:
df_final['UF'] = df_final['Estado'].map(d_uf)
df_final = df_final[['UF'] + list(df_final.columns[:-1])]

### 5.0 Visualização por Estado

In [ ]:
# df_final.sort_values(by='% pessoas com deficiência', ascending=False).set_index('Estado')[['% com deficiência', '% pessoas com deficiência']]

In [ ]:
### 

In [ ]:
df_pop_mun = pd.read_excel('..\\hackathon-cer-brasil-2021\\cleiton\\pop_mun_ibge_2010.xlsx')
df_pop_mun = df_pop_mun[['uf', 'cod_mun', 'mun', 'pop_abs', 'pop_rel_uf']]
df_pop_mun.columns = ['UF', 'Código do Município', 'Município',  'População Total', 'População Relativa']

In [ ]:
df_municipios = df_pop_mun.merge(df_final, how='left', on='UF')

In [ ]:
df_municipios['qtd. pessoas c/ deficiencia'] = round(df_municipios['População Relativa'] * df_municipios['Pelo menos uma'])
df_municipios['% pessoas c/ deficiencia'] = round((df_municipios['População Relativa'] * df_municipios['Pelo menos uma'])/df_municipios['População Total'],4)*100

In [ ]:
df_municipios[['UF', 'Município', 'População Total', 'qtd. pessoas c/ deficiencia', '% pessoas c/ deficiencia']].sort_values(by='% pessoas c/ deficiencia', ascending=False).head()

In [ ]:
df_final

In [ ]:
df_pop_mun.head()

In [ ]:
df_pop_mun[df_pop_mun['UF']=='AC']#['População Relativa']

In [ ]:
total_deficientes_acre = 165892
((df_pop_mun[df_pop_mun['UF']=='AC']['População Relativa'] * 
  total_deficientes_acre)/733558*100)

In [ ]:
total_deficientes_acre = 165892
((df_pop_mun[df_pop_mun['UF']=='AC']['População Relativa'] * 
  total_deficientes_acre)/190755799*100)

In [ ]:
pop_bras_2010 = 190755799
df_pop_mun.columns = ['UF', 'c_cod_mun', 'c_mun', 'c_pop_abs', 'c_pop_rel_uf']
df_all = pd.merge( df_pop_mun, df_final, how='left', on='UF' )
df_all.loc[:, 'c_%_pop_def_rel_uf'] = df_all['c_pop_rel_uf'] * df_all['Pelo menos uma'] / df_all['Total'] *100
df_all.loc[:, 'c_%_pop_def_rel_bras'] = df_all['c_pop_rel_uf'] * df_all['Pelo menos uma'] / pop_bras_2010 *100

In [ ]:
df_all.to_csv( 'df_all.csv', index=False )

In [ ]:
df_all.sample(5)

In [ ]:
df_all.sort_values( 'c_%_pop_def_rel_bras', ascending=False ).head(10)

## Unindo aos dados do IBP

In [ ]:
df_ibp = pd.read_csv( '../hackathon-cer-brasil-2021/cleiton/IBP/BDI_Municipalities-Level_Short.csv' )
df_ibp.sample(5)

In [ ]:
# Checando se o código de município é o mesmo entre os 2 dataframes
df_ibp[df_ibp['cod_municipio'].isin([2202729,4200309,3200706,3154804,3143906])]

In [ ]:
df_ibp = df_ibp.drop( columns=['uf', 'nome_da_uf', 'codmun6', 'nome_do_municipio'] )
df_ref = pd.merge( df_all, df_ibp, how='left', left_on='c_cod_mun', right_on='cod_municipio' )
df_ref = df_ref.rename( columns={'measure_1f_12': 'ibp', 'd_measure_1f_12': 'd_ibp', 'q_measure_1f_12': 'q_ibp' } )
df_ref.drop( columns=['cod_municipio'], inplace=True )
df_ref.sample(5)

In [ ]:
df_mun = pd.read_csv( 'municipios.csv' )
df_mun.sample(5)

In [ ]:
df_mun = df_mun.drop( columns=['nome', 'capital', 'codigo_uf', 'siafi_id', 'ddd', 'fuso_horario'] )
df_mun.columns = ['c_cod_mun', 'lat', 'long']
df_mun['lat_long'] = df_mun['lat'].astype( str ) + ',' + df_mun['long'].astype( str )
df_mun.drop( columns=['lat', 'long'], inplace=True )
df_ref = pd.merge( df_ref, df_mun, how='left', on='c_cod_mun' )
df_ref

In [ ]:
df_ref.to_csv( 'df_all_ibp.csv', index=False )